In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
import yfinance as yf
from datetime import datetime

def calculate_strategy(df):
    # Calculate RSI and Bollinger Bands
    df['RSI'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    df['bb_up'], df['bb_down'] = ta.volatility.BollingerBands(df['Close']).bollinger_mavg(window=20, window_dev=2)

    # Buy and sell signals
    df['Signal'] = np.where((df['RSI'] < 30) & (df['Close'] > df['bb_down']), 1, 
                            np.where((df['RSI'] > 70) & (df['Close'] < df['bb_up']), -1, 0))
    df['Position'] = np.where(df['Signal'] == 1, 1, np.where(df['Signal'] == -1, -1, np.nan))
    df['Position'].fillna(method='ffill', inplace=True)
    df['Position'].fillna(0, inplace=True)

    # Calculate PnL
    df['PnL'] = (df['Close'] - df['Close'].shift(1)) * df['Position']
    df['CumPnL'] = df['PnL'].cumsum()
    
    return df

# Load data from Yahoo Finance
ticker = "BTC-USD"
end_date = datetime.today().strftime("%Y-%m-%d")
df = yf.download(ticker, start="2017-01-01", end=end_date)
df = df[['Close']]

# Calculate strategy
df = calculate_strategy(df)

# Plot PnL
plt.plot(df['CumPnL'])
plt.xlabel('Date')
plt.ylabel('Cumulative PnL')
plt.title('Strategy PnL')
plt.show()

# Print signal table
print(df[['Close', 'RSI', 'bb_down', 'bb_up', 'Signal']].tail(10))


[*********************100%***********************]  1 of 1 completed


AttributeError: module 'talib' has no attribute 'momentum'